In [5]:
from gym.wrappers import GrayScaleObservation, ResizeObservation, RecordVideo, TimeLimit
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage, VecMonitor
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import RIGHT_ONLY
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
from stable_baselines3 import PPO
import gym_super_mario_bros
import os

#from TestScript import MarioRewardWrapper  # tu wrapper personalizado

In [6]:
def make_recording_env(rank: int = 0, video_folder: str = "../videos"):
    # Crea carpeta de vídeo
    os.makedirs(video_folder, exist_ok=True)
    # Entorno base
    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
    env = JoypadSpace(env, RIGHT_ONLY)
    # Grabación en tiempo real (antes de frame-skip)
    env = RecordVideo(env, video_folder, episode_trigger=lambda idx: True)
    # Luego wrappers idénticos al entrenamiento
    env = MaxAndSkipEnv(env, skip=4)
    #env = MarioRewardWrapper(env, factor=0.03, goal_bonus=100, death_penalty=-50, max_steps=10_000)
    env = GrayScaleObservation(env, keep_dim=True)
    env = ResizeObservation(env, (84, 84))
    env.seed(42 + rank)
    return env

In [7]:
model = PPO.load("../models/checkpoints/mario_ppo_6100000_steps.zip")

# DummyVecEnv para evaluación (un solo env con grabación)
record_env = DummyVecEnv([lambda: make_recording_env(rank=1, video_folder="../videos")])
# Monitorea episodios vectorizados correctamente
record_env = VecMonitor(record_env)
# Aplica frame-stack y transpose para el modelo
record_env = VecFrameStack(record_env, n_stack=4, channels_order='last')
record_env = VecTransposeImage(record_env)

c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\stable_baselines3\common\save_util.py:435: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_

In [8]:
obs = record_env.reset()
for _ in range(10_000):  # suficientes pasos para un episodio
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = record_env.step(action)
    if done:
        obs = record_env.reset()

record_env.close()